# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "output_data/cities.csv"
city_df = pd.read_csv(output_data_file)
city_df.head()

,Unnamed: 0,City,Lat,Lng,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,itarema,-2.9248,-39.9167,BR,81.64,78.0,100.0,4.41,1649769675
1,1,port alfred,-33.5906,26.8910,ZA,68.18,82.0,100.0,5.99,1649769860
2,2,avarua,-21.2078,-159.7750,CK,78.85,89.0,75.0,4.61,1649769860
3,3,hatillo,18.4863,-66.8254,PR,83.79,74.0,6.0,7.00,1649769861
4,5,cidreira,-30.1811,-50.2056,BR,72.55,82.0,100.0,5.77,1649769861


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Bring in location data pairs, 
locations = city_df[["Lat", "Lng"]]

#humidity data, 
humidity = city_df['Humidity']

#define map 
fig = gmaps.figure()

#and layer, and add layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example: 
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

ideal_city_df = city_df[city_df['Cloudiness']<10]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp']>60]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp']<75]
ideal_city_df = ideal_city_df[ideal_city_df['Wind Speed']<20]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity']<60]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity']>30]
ideal_city_df = ideal_city_df.dropna()
ideal_city_df


,Unnamed: 0,City,Lat,Lng,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date
40,42,emerald,-23.5333,148.1667,AU,74.34,52.0,0.0,9.69,1649769731
47,49,rosetta,31.4044,30.4164,EG,62.55,55.0,0.0,18.41,1649769874
91,98,yairipok,24.6667,94.0667,IN,73.02,53.0,0.0,3.20,1649769889
99,106,lukavica,44.7628,18.1689,BA,63.90,37.0,9.0,4.03,1649769892
165,179,apricena,41.7839,15.4437,IT,66.29,47.0,9.0,7.49,1649769914
200,216,saint-francois,46.4154,3.9054,FR,69.67,38.0,0.0,12.66,1649769925
223,241,marsa matruh,31.3525,27.2453,EG,66.40,45.0,0.0,18.41,1649769933
358,391,maldonado,-34.6667,-54.9167,UY,65.59,56.0,0.0,9.31,1649769978
384,418,pombia,45.6530,8.6336,IT,67.32,33.0,0.0,6.91,1649769986
398,433,valparaiso,-33.0393,-71.6273,CL,61.03,58.0,0.0,6.91,1649769991


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create dataframe and empty column
hotel_df = ideal_city_df[['City', 'Country','Lat', 'Lng']]
hotel_df = hotel_df.reindex(columns = ['Hotel Name','City', 'Country', 'Lat', 'Lng'])
hotel_df = hotel_df.reset_index(drop = True)

hotel_df


,Hotel Name,City,Country,Lat,Lng
0,NaN,emerald,AU,-23.5333,148.1667
1,NaN,rosetta,EG,31.4044,30.4164
2,NaN,yairipok,IN,24.6667,94.0667
3,NaN,lukavica,BA,44.7628,18.1689
4,NaN,apricena,IT,41.7839,15.4437
5,NaN,saint-francois,FR,46.4154,3.9054
6,NaN,marsa matruh,EG,31.3525,27.2453
7,NaN,maldonado,UY,-34.6667,-54.9167
8,NaN,pombia,IT,45.6530,8.6336
9,NaN,valparaiso,CL,-33.0393,-71.6273


In [7]:
#Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_type = "lodging"
radius = 5000
params = {
    "location": "",
    "type": target_type,
    "radius": radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        print("oops, something went wrong with the API call")
        #print(response)

hotel_df

#Plot the hotels on top of the humidity heatmap, with each pin containing the Hotel Name, City, and Country, as in the following image:


,Hotel Name,City,Country,Lat,Lng
0,The Irish Village,emerald,AU,-23.5333,148.1667
1,Green Valley,rosetta,EG,31.4044,30.4164
2,Ningthounai,yairipok,IN,24.6667,94.0667
3,DUJA,lukavica,BA,44.7628,18.1689
4,Bed & Breakfast CORSO ROMA,apricena,IT,41.7839,15.4437
5,Manoir Du Bonheur,saint-francois,FR,46.4154,3.9054
6,Beau Site,marsa matruh,EG,31.3525,27.2453
7,La Cimarrona,maldonado,UY,-34.6667,-54.9167
8,Albergo Fontana,pombia,IT,45.6530,8.6336
9,Hotel Ultramar,valparaiso,CL,-33.0393,-71.6273


In [11]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# Create marker layer
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))